In [ ]:
import os
import ee
import geemap

In [ ]:
os.getcwd()

In [ ]:
folder='G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3'
os.chdir(folder)

In [ ]:
os.getcwd()

In [ ]:
Map = geemap.Map(location=[50.99912774142991,-123.24266622024662], zoom_start=6)
Map

In [ ]:
fp=ee.FeatureCollection('projects/ee-luizmardeab/assets/Vancouver_Island')

In [ ]:
# Load the ALOS PALSAR dataset for the year 2018
dataset = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH') \
    .filter(ee.Filter.date('2018','2020'))

# Select the HH and HV bands
sar_hh_hv = dataset.select(['HH', 'HV'])

# Visualization parameters for the HH band
sar_hh_vis = {
    'bands': ['HH'],
    'min': 0.0,
    'max': 10000.0,
}

# Define the region of interest (Vancouver Island)
region = ee.Geometry.Polygon([
    [-129.15, 48.25],
    [-122.5, 48.25],
    [-122.5, 50.93],
    [-129.15, 50.93],
    [-129.15, 48.25]
])

# Calculate the mean image and clip to the region of interest
sar_hh_clipped = sar_hh_hv.median().clip(region)

# Create a map using geemap
Map = geemap.Map(center=[49.6, -125.5], zoom=7)
Map.addLayer(sar_hh_clipped, sar_hh_vis, 'SAR HH (Clipped)')

# Display the map
Map

In [ ]:
s2Sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')

START_DATE = ee.Date('2019-06-01')
END_DATE = ee.Date('2019-10-01')
MAX_CLOUD_PROBABILITY = 30


def maskClouds(img):
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return img.updateMask(isNotCloud)

In [ ]:
# Example asset that needs this operation:
# COPERNICUS/S2_CLOUD_PROBABILITY/20190301T000239_20190301T000238_T55GDP

def maskEdges(s2_img):
    return s2_img.updateMask(
        s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))

# Filter input collections by desired data range and region.
criteria = ee.Filter.And(
    ee.Filter.bounds(region),
    ee.Filter.date(START_DATE, END_DATE)
)

s2Sr = s2Sr.filter(criteria).map(maskEdges)
s2Clouds = s2Clouds.filter(criteria)

# Join S2 SR with cloud probability dataset to add cloud mask.
s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
    primary=s2Sr,
    secondary=s2Clouds,
    condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
)

s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskClouds).median()
s2CloudMasked=s2CloudMasked
rgbVis = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}


ndwi = s2CloudMasked.normalizedDifference(['B3', 'B8']).rename('NDWI')

ndvi = s2CloudMasked.normalizedDifference(['B3', 'B5']).rename('NDVI')

ndviThreshold = ndvi.lte(0)
ndviMask = ndviThreshold.updateMask(ndviThreshold)

trueColorViz = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 2700,
    'gamma': 1.3
}

In [ ]:
# Create a map using geemap
Map = geemap.Map(center=[49.6, -125.5], zoom=8)
Map.addLayer(s2CloudMasked, trueColorViz, 'SAR HH (Clipped)')

# Display the map
Map

In [ ]:
#// Which bands to select:L8
bandNumbers = [1,2,3,7,10,11,4,5,6,8];
bandNames = ee.List(['blue','green','red','nir','swir1','swir2',"redge1","redge2","redge3","redge4"]);
#// apply over the image collection
l8collection = ee.ImageCollection(s2SrWithCloudMask).map(maskClouds).select(bandNumbers, bandNames)


# Reduce the l8collection using the reducers
s2 = l8collection.median()
s2

In [ ]:
# # Create image collection of S-1 images filtered by date, polarization, resolution, and orbit
imgVV = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filterBounds(region) \
    .select('VV')


desc = imgVV.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = imgVV.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

summer = ee.Filter.date('2019-06-01', '2019-10-01')

descChangeVV = desc.filter(summer).mean().float().rename('ascVV')

ascChangeVV = asc.filter(summer).select('VV').mean().float().rename('descVV')

imgVH = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filterBounds(region) \
    .select('VH')

desc = imgVH.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = imgVH.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

ascChangeVH = asc.filter(summer).mean().float().rename('ascVH')

descChangeVH = desc.filter(summer).mean().float().rename('descVH')

s1 = ascChangeVH

s1 = s1.addBands([ascChangeVV, descChangeVH, descChangeVV])

In [ ]:
s1 

In [ ]:
#Variables
#raw
gdem = ee.Image("projects/sat-io/open-datasets/ASTER/GDEM").float().rename('DEM')
longitude = ee.Image.pixelLonLat().select('longitude').float()
latitude = ee.Image.pixelLonLat().select('latitude').float()



In [ ]:
vars = s2.addBands([s1,sar_hh_clipped,gdem,longitude,latitude])

In [ ]:
imgMasked = vars
imgMasked

In [ ]:
chm = ee.Image('projects/ee-luizmardeab/assets/chm_VI').float().rename('chm')

In [ ]:
trueColorViz = {
    'bands': ['red', 'green', 'blue'],
    'min': 0,
    'max': 2700,
    'gamma': 2.5
}

VH_viz = {
    'bands': ['ascVH'],
    'min': -25,  # You can adjust these values based on the data range
    'max': 0,
    'gamma': 1.5  # A moderate gamma correction
}
places =ee.Geometry.Polygon([[-129.15,48.25],
[-123,48.25],
[-123,50.93],
[-129.15,50.93],
[-129.15,48.25]])

places_pred =ee.Geometry.Polygon([[-129.25,48.15],
[-122.9,48.25],
[-122.9,51.03],
[-129.25,51.03],
[-129.25,48.15]])

grid = geemap.fishnet(places, rows=11, cols=12,delta=0.5)
grid_pred = geemap.fishnet(places_pred, rows=11, cols=12,delta=0.25)
style = {'color': '000000ff', 'width': 2, 'lineType': 'solid'}
Map = geemap.Map(center=[49.6, -125.5], zoom=7)

#Add the elevation model to the map object.
Map.addLayer(imgMasked, VH_viz, 'VH')
Map.addLayer(imgMasked, trueColorViz, 'Age')
Map.addLayer(chm, {'bands': ['chm'],'min': 0,'max': 100}, 'Height')
# Map.addLayer(grid_pred, style, 'Grids-Pred')
Map.addLayer(grid, style, 'Grids-Full')
Map



In [ ]:
folder='G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3'
os.chdir(folder)

In [ ]:
# Specify the path of the directory
out_dir = folder + '/Images_VSG'
out_dir

try:
    # Create target directory if it doesn't exist
    os.makedirs(out_dir)
    print(f"Directory '{out_dir}' created successfully.")
except FileExistsError:
    print(f"Directory '{out_dir}' already exists.")
except OSError as e:
    print(f"Error creating directory '{out_dir}': {e}")

In [ ]:
# imgMasked = imgMasked.reproject(crs="EPSG:3005", scale=10)

In [ ]:
geemap.download_ee_image_tiles(
    imgMasked, grid, out_dir, prefix="Sentinel_", crs="EPSG:3005", scale=10,overwrite=True,resampling='bicubic'
)

In [ ]:
# geemap.ee_to_shp(grid, filename='G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3/tiles.shp')

# Crop mask to tile extent

In [ ]:
import geopandas as gpd
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.mask import mask
from rasterio.enums import Compression
from shapely.geometry import mapping
import geemap
import os
import numpy as np
import glob

In [ ]:
root_directory = 'G:\\Conservation Solution Lab\\People\\Luizmar\\PhD_Luizmar\\Chapter_3'
os.chdir(root_directory)

In [ ]:
#Resample Mask to match sentinel data
def resample_to_match(src,band_data, ref_raster_path, dst_path):
    with rasterio.open(ref_raster_path) as ref:
        ref_transform = ref.transform
        ref_width = ref.width
        ref_height = ref.height
        ref_crs = ref.crs
        
        kwargs = src.meta.copy()
        kwargs.update({
            'count': 1,
            'crs': ref_crs,
            'transform': ref_transform,
            'width': ref_width,
            'height': ref_height
        })

        with rasterio.open(dst_path, 'w', **kwargs) as dst:
            reproject(
                source=band_data,
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=ref_transform,
                dst_crs=ref_crs,
                resampling=Resampling.nearest
            )
            print(f"Mask save to {dst_path}")

In [ ]:
# Specify the path of the directory
directory = os.getcwd()
out_dir = directory+ "/Masks_2"
# Use try-except block to handle potential errors
try:
    # Create target directory if it doesn't exist
    os.makedirs(out_dir)
    print(f"Directory '{out_dir}' created successfully.")
except FileExistsError:
    print(f"Directory '{out_dir}' already exists.")
except OSError as e:
    print(f"Error creating directory '{out_dir}': {e}")

#Reproject raster data

In [ ]:
input_raster = 'G:/reference/hght_metrics.tif'
output_raster = 'G:/reference/max_height.tif'

In [ ]:
with rasterio.open(input_raster) as src:
    print(src.crs)

In [ ]:
# Step 1: Read the Rasterio Image

# Define the source and target CRS
dst_crs = 'EPSG:3005'

# Open the source raster
def reproject_raster(src_path, dst_path, target_crs):
    with rasterio.open(src_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(dst_path, 'w', **kwargs) as dst:
             #for i in range(1, src.count + 1):
            # Read the specific band (e.g., band 8) band 8 is where the p99 is
            src_band = src.read(8)
            src_band[src_band < 0] = np.nan 
            src_band[src_band > 100] = np.nan 
            
            # Reproject the band data using nearest neighbor resampling
            reproject(
                source=src_band,
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=target_crs,
                resampling=Resampling.nearest
            )
            print(f'Reprojected raster saved to {output_raster} with projection {dst_crs}')



# Example usage:
src_raster = input_raster
dst_raster = output_raster
target_crs = 'EPSG:3005'  # Target CRS

reproject_raster(src_raster, dst_raster, target_crs)

Cropping to sentinel vars tile extent

In [ ]:
os.getcwd()

In [ ]:
# Open the original image
with rasterio.open(output_raster) as src_orig:
    # Rename band
    band_name = "z_sd"
    # Read only band 8
    band_num = 3
    band_data = src_orig.read(band_num)
    masked_data = np.where(band_data == -3.4028235e+38, np.nan, band_data)  #change np.nan to zero
    
    #Get stats for clean data
    band_data = masked_data
    min_val = np.nanmin(band_data)  # Minimum value in the band
    max_val = np.nanmax(band_data)  # Maximum value in the band
    mean_val = np.nanmean(band_data)  # Minimum value in the band
    std_val = np.nanstd(band_data)  # Maximum value in the band
    percentile_95 = np.nanpercentile(band_data, 95)
    percentile_98 = np.nanpercentile(band_data, 98)
    percentile_99 = np.nanpercentile(band_data, 99.99)
    

    print(f'Image min: {min_val}, max: {max_val}, mean: {mean_val}, std: {std_val}, '
          f'95th percentile: {percentile_95}, 98th percentile: {percentile_98}, 99th percentile: {percentile_99}')

In [ ]:
with rasterio.open(output_raster) as src_orig:
    print(src_orig.crs)

tile_path = f'G:/predictor/Sentinel_006.tif'
print(f"Opening {tile_path}")
with rasterio.open(tile_path) as src_tile:
    print(src_tile.crs)

In [ ]:
def get_extent(src):
    bounds = src.bounds
    extent = (bounds.left, bounds.bottom, bounds.right, bounds.top)
    return extent

In [ ]:
#Resample Mask to match sentinel data
from rasterio.warp import reproject, Resampling

def resample_to_match(src,band_data, ref_raster_path, dst_path):
    with rasterio.open(ref_raster_path) as ref:
        ref_transform = ref.transform
        ref_width = ref.width
        ref_height = ref.height
        ref_crs = ref.crs
        
        kwargs = src.meta.copy()
        kwargs.update({
            'count': 1,
            'crs': ref_crs,
            'transform': ref_transform,
            'width': ref_width,
            'height': ref_height
        })

        with rasterio.open(dst_path, 'w', **kwargs) as dst:
            reproject(
                source=band_data,
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=ref_transform,
                dst_crs=ref_crs,
                resampling=Resampling.nearest
            )
            print(f"Mask save to {dst_path}")


In [ ]:
numvar = [None] * 132  # Initialize a list of size 132 with None or any placeholder

for num in range(0, 132):
    numvar[num] = "{:03d}".format(num + 1)
    # print(numvar[num])

In [ ]:
src_path = 'G:/reference/max_height.tif'
with rasterio.open(src_path) as src:
    band_data = src.read(3)
    for i in numvar:  # Assuming 4x4 tiles
        # Open each tile image
        ref_raster_path = f'G:/predictor/Sentinel_{i}.tif'
        print(f"Opening {ref_raster_path}")
        
        # Write the cropped image to a new file
        dst_path = os.path.join(output_dir, f'G:/reference/Masks_2/Mask_{i}.tif')
    
        #Crop Mask to Image extent so they align perfectly
        resample_to_match(src,band_data,ref_raster_path, dst_path)

In [ ]:
 global_99p_values